In [ ]:
include("../src/PanTaGruEl.jl")
using .PanTaGruEl

In [ ]:
source_folder = "../../sources"
out_folder = "../../tmp"

In [ ]:
grid_data = load_gridkit_data(source_folder)
retrieve_country!(source_folder, grid_data, 50)
differentiate_line_and_dc_cable!(grid_data)
remove_gen_duplicate!(grid_data)
assign_simple_type_to_gen!(grid_data)
differentiate_gen_and_renew!(grid_data)
discard_non_trans!(grid_data)

In [ ]:
country = ["AL", "AT", "BA", "BE", "BG", "CH", "CZ", "DE", "ES",
    "FR", "GR", "HR", "HU", "IL", "IT", "LU", "ME", "MK",
    "NL", "PL", "PT", "RO", "RS", "SA", "SI", "SK", "DK"]

#append!(country,["TR", "UA"]

scenario = extract_grid_from_country(grid_data, country, start = "country" => "CH")

get_line_parameters!(scenario)
get_trans_parameters!(scenario)

remove_neg_gen!(scenario)
crosscheck_with_wri!(source_folder, scenario, dthres = 50)
large_gen_on_380!(scenario)
allow_full_gen!(scenario)
assign_inertia_constant!(scenario)
assign_marginal_cost!(scenario)
distribute_population!(source_folder, scenario, region="Europe")
assign_active_demand!(scenario, national_demand("winter_peak"))
assign_bus_type!(scenario)

In [ ]:
println("Total gen: $(sum(scenario["gen"].capacity)), Total demand: $(sum(scenario["demand"].active))")

In [ ]:
add_french_renew!(source_folder, scenario)
add_german_renew!(source_folder, scenario)

In [ ]:
reinforce_network!(scenario)

In [ ]:
export_csv("$out_folder/Europe", scenario)
export_matpower("$out_folder/Europe/europe.m", scenario)